## 1. Introduction: Business Problem Statement ##
A young French couple recently moved from Paris to Chicago and would like to open a restaurant to showcase famous French dishes. They are confident in their culinary skills and convinced that their new business venture will be a success if they can find a great location for their restaurant. They hired John, a Chicago native who recently open a marketing consulting business after working for a large analytics consulting firm for several years. The young couple would like the restaurant to meet the following requirements:

1. Target customers: tourists, business employees and local residents who are looking for delicious lunch or dinners
2. Private residence: since they are new to the city they would like to reside in a safe neighborhood they can return to after long hours at the restaurant.
3. Commute: they will prefer a short commute since they will be spending most of their day at the restaurant.
4. School: they also have a young child and would like him to attend an elementary school with good safety and college enrollment records
5. Deleverables: they would like John to recommend a list of three locations on a Chicago map that meet the above four requirements


## 2. Data Description ##
To find the best locations that satisfy the young french business requirements, John plans to use the following three datasets:

1. Chicago Criminality: this dataset includes information about crimes that occurred in Chicago neighborhoods between 2001 and 2018 as well as details such as type of crimes, neighborhoods where the crimes took place and more.John plans to use these data to help locate safe neighborhoods.
2. Chicago Schools: it includes information about Chicago public schools in terms of safety, college preparation, neighborhoods and more. John plans to use these data to help locate safe schools for the couple's child.
3. Chicago Coordinates: it includes Chicago neighborhoods’ geospatial data; they are not included in the above two files. This dataset also includes reviews, tips and information about popular Chicago’s venues and attractions. John plans to use these data to help locate popular venues and attractions.


## 3. Methodology ##

In [ ]:
# The first step for John is to collect data relevant to the couple's requirements

In [1]:
# let's import pandas for data analysis
import pandas as pd

# Let's install the sql module to start retrieving data needed for the analysis
!pip install ipython-sql

# load the sql extension after the module installation
%load_ext sql

print('Sql extension successully loaded')

     |████████████████████████████████| 42 kB 2.3 MB/s  eta 0:00:01
  Created wheel for prettytable: filename=prettytable-0.7.2-py3-none-any.whl size=13698 sha256=2d7c8f328f304cc864c7006b6541b24b950561a3807404d1e9097fb060288289
  Stored in directory: /tmp/wsuser/.cache/pip/wheels/8c/76/0b/eb9eb3da7e2335e3577e3f96a0ae9f74f206e26457bd1a2bc8
Successfully built prettytable
Sql extension successully loaded


In [2]:
# let's connect to the database with valid credentials
cnxn = %sql ibm_db_sa://zqm51554:998-8p6qldltnxbd@dashdb-txn-sbox-yp-dal09-11.services.dal.bluemix.net:50000/BLUDB      
print('Connection successful')

Connection successful


In [3]:
# A Chicago native, John has collected crimes and schools data about the city over the years and stored them in his database; let's retrieve them.
%sql select TABSCHEMA, TABNAME, CREATE_TIME from SYSCAT.TABLES where TABNAME in ('CHICAGO_CRIME_DATA', 'SCHOOLS')

 * ibm_db_sa://zqm51554:***@dashdb-txn-sbox-yp-dal09-11.services.dal.bluemix.net:50000/BLUDB
Done.


tabschema,tabname,create_time
ZQM51554,SCHOOLS,2020-11-14 21:43:42.289882
ZQM51554,CHICAGO_CRIME_DATA,2020-11-14 22:46:20.918391


In [4]:
# create connection for pandas in order to read these tables into pandas dataframe
import ibm_db
import ibm_db_dbi
conn = ibm_db.connect("DATABASE=BLUDB;HOSTNAME=dashdb-txn-sbox-yp-dal09-11.services.dal.bluemix.net; PORT=50000; PROTOCOL=TCPIP; UID=zqm51554; PWD=998-8p6qldltnxbd;","","")
pconn = ibm_db_dbi.Connection(conn)

In [5]:
#The CHICAGO_CRIMES_DATA table can help locate safe neighborhoolds for the restaurant location.let's examine that table: 

%sql select colname,typename, length from SYSCAT.columns where tabname = 'CHICAGO_CRIME_DATA' 

 * ibm_db_sa://zqm51554:***@dashdb-txn-sbox-yp-dal09-11.services.dal.bluemix.net:50000/BLUDB
Done.


colname,typename,length
ID,INTEGER,4
CASE_NUMBER,VARCHAR,8
DATE,VARCHAR,16
BLOCK,VARCHAR,35
IUCR,VARCHAR,4
PRIMARY_TYPE,VARCHAR,33
DESCRIPTION,VARCHAR,46
LOCATION_DESCRIPTION,VARCHAR,33
ARREST,VARCHAR,5
DOMESTIC,VARCHAR,5


In [6]:
# show the crime table with only select columns of interest
crime = pd.read_sql("select * from CHICAGO_CRIME_DATA ", pconn) #read the CRIME table into a pandas dataframe  

crime_feature = crime[["DISTRICT","COMMUNITY_AREA_NUMBER","ID","PRIMARY_TYPE", "ARREST", "LOCATION_DESCRIPTION"]] # let's retain only select columns of interest

crime_features = crime_feature[crime_feature["COMMUNITY_AREA_NUMBER"] > 0] # let's remove missing or invalid community area numbers

crime_features.rename(columns = {'ID':'Crime_ID', 'PRIMARY_TYPE':'Crime_TYPE'}, inplace = True) # rename these columns for convenience 

print ('The sizes of the original and sliced Crime datasets are:', crime.shape,  crime_features.shape)
                      
crime_features.head()

The sizes of the original and sliced Crime datasets are: (533, 22) (490, 6)


/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/pandas/core/frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,DISTRICT,COMMUNITY_AREA_NUMBER,Crime_ID,Crime_TYPE,ARREST,LOCATION_DESCRIPTION
0,9,58.0,3512276,THEFT,FALSE,SMALL RETAIL STORE
1,11,23.0,3406613,THEFT,FALSE,OTHER
2,2,38.0,8002131,THEFT,FALSE,NURSING HOME/RETIREMENT HOME
3,4,46.0,7903289,THEFT,FALSE,RESIDENCE
4,8,66.0,10402076,THEFT,FALSE,ALLEY


In [7]:
# 1. let's locate the top 10 safest districts  
%sql select DISTRICT, count(ID) AS TOTAL_Crime \
   from CHICAGO_CRIME_DATA \
   group by DISTRICT \
   order by TOTAL_Crime asc \
   nulls last limit 10

 * ibm_db_sa://zqm51554:***@dashdb-txn-sbox-yp-dal09-11.services.dal.bluemix.net:50000/BLUDB
Done.


district,total_crime
20,9
22,10
24,10
1,14
5,16
14,17
18,17
17,17
16,19
2,21


In [23]:
# The next step is to locate community areas with the best elementary schools in terms of safety and college enrollment records 

In [8]:
# John decides to examine the SCHOOLS table 
%sql select colname,typename, length from SYSCAT.columns where tabname = 'SCHOOLS'

 * ibm_db_sa://zqm51554:***@dashdb-txn-sbox-yp-dal09-11.services.dal.bluemix.net:50000/BLUDB
Done.


colname,typename,length
Track_Schedule,VARCHAR,12
CPS_Performance_Policy_Status,VARCHAR,16
CPS_Performance_Policy_Level,VARCHAR,15
HEALTHY_SCHOOL_CERTIFIED,VARCHAR,3
Safety_Icon,VARCHAR,11
SAFETY_SCORE,SMALLINT,2
Family_Involvement_Icon,VARCHAR,11
Family_Involvement_Score,VARCHAR,3
Environment_Icon,VARCHAR,11
Environment_Score,SMALLINT,2


In [9]:
# read the SCHOOLS table into a pandas dataframe based on select features of interest 
schl = pd.read_sql("select * from SCHOOLS ", pconn) 

schl_features = schl[["Police_District", "COMMUNITY_AREA_NUMBER","COMMUNITY_AREA_NAME","School_ID", "NAME_OF_SCHOOL", "Elementary, Middle, or High School", "COLLEGE_ENROLLMENT", "SAFETY_SCORE", "Longitude", "Latitude"]] 

schl_features.rename(columns = {'Elementary, Middle, or High School':'School_Type', 'Police_District':'DISTRICT'}, inplace = True) # rename these columns in the SCHOOLS data

print ('The sizes of the original and sliced School datasets are:', schl.shape,  schl_features.shape)
print('Chicago has {} uniques Districts.'.format(len(schl_features['DISTRICT'].unique())))
print('Chicago has {} uniques Communities.'.format(len(schl_features['COMMUNITY_AREA_NUMBER'].unique())))
schl_features.head()

The sizes of the original and sliced School datasets are: (566, 78) (566, 10)
Chicago has 23 uniques Districts.
Chicago has 77 uniques Communities.


/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/pandas/core/frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,DISTRICT,COMMUNITY_AREA_NUMBER,COMMUNITY_AREA_NAME,School_ID,NAME_OF_SCHOOL,School_Type,COLLEGE_ENROLLMENT,SAFETY_SCORE,Longitude,Latitude
0,18,7,LINCOLN PARK,610038,Abraham Lincoln Elementary School,ES,813,99.0,-87.644522,41.924497
1,4,43,SOUTH SHORE,610281,Adam Clayton Powell Paideia Community Academy ...,ES,521,54.0,-87.556736,41.760324
2,8,70,ASHBURN,610185,Adlai E Stevenson Elementary School,ES,1324,61.0,-87.731702,41.747111
3,9,61,NEW CITY,609993,Agustin Lara Elementary Academy,ES,556,56.0,-87.672145,41.809757
4,9,34,ARMOUR SQUARE,610513,Air Force Academy High School,HS,302,49.0,-87.632794,41.828146


In [10]:
# let's locate the 10 best elementary scools in terms of college enrollment
Schl_enroll = schl_features[['DISTRICT','COLLEGE_ENROLLMENT']]\
.groupby('DISTRICT').agg(['mean', 'count'])\
.sort_values(by=('COLLEGE_ENROLLMENT','mean'), ascending=False) # sort from the highest enrollment to the lowest

Schl_enroll.head(10)

COLLEGE_ENROLLMENT      
                       mean count
DISTRICT                         
8               1020.780488    41
25               990.517241    29
17               849.461538    26
20               839.583333    12
16               797.285714    28
24               771.214286    14
19               734.444444    27
18               688.000000    12
9                683.135135    37
12               665.294118    17

In [11]:
# let's locate the 10 best elementary scools in terms of safety 
Schl_safety = schl_features[['DISTRICT','SAFETY_SCORE']]\
.groupby('DISTRICT').agg(['mean', 'count'])\
.sort_values(by=('SAFETY_SCORE','mean'), ascending=False) # sort from the safest to the least safest

Schl_safety.head(10)

SAFETY_SCORE      
                 mean count
DISTRICT                   
18          74.300000    10
19          73.250000    24
16          68.576923    26
1           67.142857     7
17          64.250000    24
24          60.583333    12
13          60.277778    18
20          59.583333    12
14          55.388889    18
12          54.312500    16

In [34]:
# let's close the connection
ibm_db.close (conn)

print('Connection successfully closed')

Connection successfully closed


Result 1: the review of the top 10 districts in terms of criminality, college enrollment and school safety shows that the District 1 appears the most among the top 5 districts. He also knows that this district is close to the couple's current residence. Therefore John decided to focus on District 1 in order to narrow its search for the restaurant location.

In [ ]:
# First let's import the geospatial data to show Chicago's communities on a map #

In [12]:
#Import necessary libraries for geospatial data an visualization
import numpy as np 
import random 
!pip install geopy
from geopy.geocoders import Nominatim # to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library: 
from pandas.io.json import json_normalize

# plotting folium library
! pip install folium==0.5.0

import folium 
print ('Folium installed')
print ('Libraries imported.')

     |████████████████████████████████| 79 kB 6.5 MB/s eta 0:00:011
  Created wheel for folium: filename=folium-0.5.0-py3-none-any.whl size=76240 sha256=673d40fb2de16aa2fe9763ec80e0558dd17f3ddd178c0506e3bfe8bf39ba2243
  Stored in directory: /tmp/wsuser/.cache/pip/wheels/b2/2f/2c/109e446b990d663ea5ce9b078b5e7c1a9c45cca91f377080f8
Successfully built folium
Folium installed
Libraries imported.


In [13]:
# declare foursquare credentials
CLIENT_ID = 'LGVTLHELVATSOFYTF53NXJVJLQA341JV3LAYBJ4045OC1QOX' 
CLIENT_SECRET = 'PJN5FPN5OD3M2CLRPGIIN5LWBY3DS3HSOUIB2K5S10L3FSKT' 
ACCESS_TOKEN = '3FYNICLJ5WYKDKUIEEUOHM2R4V2DYBAE0AWPVNK3HX0BU3NB' 
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: LGVTLHELVATSOFYTF53NXJVJLQA341JV3LAYBJ4045OC1QOX
CLIENT_SECRET:PJN5FPN5OD3M2CLRPGIIN5LWBY3DS3HSOUIB2K5S10L3FSKT


In [14]:
# show the geospatial coordinates for Chicago
address = 'Chicago, IL'
geolocator = Nominatim(user_agent="chi_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Chicago are', latitude,longitude)

The geograpical coordinate of Chicago are 41.8755616 -87.6244212


In [15]:
# show Chicago and its community areas on a map
map_chicago = folium.Map(location=[latitude, longitude], zoom_start=10) # create map of Chicago using latitude and longitude values

# add markers to map
for lat, lng, COMMUNITY_AREA_NUMBER, DISTRICT in zip(schl_features['Latitude'], schl_features['Longitude'], schl_features['COMMUNITY_AREA_NUMBER'], schl_features['DISTRICT']):
    label = '{}, {}'.format(COMMUNITY_AREA_NUMBER, DISTRICT)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_chicago)  

map_chicago

In [ ]:
# Now, let's locate the communities with the most venues in District 1 

In [16]:
# select district1 features
District1_features = schl_features[['DISTRICT','COMMUNITY_AREA_NUMBER', 'COMMUNITY_AREA_NAME', 'School_ID','Longitude', 'Latitude']]
District1 = District1_features[District1_features.DISTRICT==1]
print(District1.shape)
District1.head()

(8, 6)


,DISTRICT,COMMUNITY_AREA_NUMBER,COMMUNITY_AREA_NAME,School_ID,Longitude,Latitude
267,1,35,DOUGLAS,609894,-87.617768,41.843794
396,1,33,NEAR SOUTH SIDE,610231,-87.629281,41.852773
428,1,35,DOUGLAS,609676,-87.617673,41.840188
446,1,33,NEAR SOUTH SIDE,609769,-87.625279,41.849697
483,1,33,NEAR SOUTH SIDE,609990,-87.628651,41.866856


In [17]:
# Let's request the json data within 500 meters radius of District 1 communities' geospatial data 
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define search radius
community_latitude = District1.loc[267,'Latitude']
community_longitude = District1.loc[267,'Longitude'] 
community_name = District1.loc[267,'COMMUNITY_AREA_NAME']

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
CLIENT_ID, 
CLIENT_SECRET, 
VERSION, 
community_latitude,
community_longitude, 
radius, 
LIMIT)
url

# Send the GET request for the first community in District 1 table above
import requests

results = requests.get(url).json()

In [18]:
# let's define a function that extracts the category of the venue using the first district in the above table
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']
    
# Clean the json file and structure it into a dataframe.
venues = results['response']['groups'][0]['items']
nearby_venues = pd.json_normalize(venues) 

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row: 
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns: 
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]
nearby_venues.head()
  
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0])) # show how many venues were returned by Foursquare

7 venues were returned by Foursquare.


In [19]:
# Explore communities in District 1:
# Let's first create a function to repeat the above task to all the communities in District1
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)

# create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format( CLIENT_ID, CLIENT_SECRET, VERSION, lat, lng, radius, LIMIT)

# make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']

# return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'], 
            v['venue']['categories'][0]['name']) for v in results])
    
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Community Name', 'Community Latitude', 'Community Longitude', 'Venue', 'Venue Latitude', 'Venue Longitude','Venue Category']

    return(nearby_venues)

In [20]:
# Now let's read the results of the above aggregation task into a new dataframe called District1_venues
District1_venues = getNearbyVenues(names=District1['COMMUNITY_AREA_NAME'],latitudes=District1['Latitude'],longitudes=District1['Longitude'])

print(District1_venues.shape)
District1_venues.head()

DOUGLAS
NEAR SOUTH SIDE
DOUGLAS
NEAR SOUTH SIDE
NEAR SOUTH SIDE
LOOP
DOUGLAS
DOUGLAS
(352, 7)


,Community Name,Community Latitude,Community Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,DOUGLAS,41.843794,-87.617768,D3: Dre's Diesel Dome,41.845420,-87.622887,Gym / Fitness Center
1,DOUGLAS,41.843794,-87.617768,Dunbar Park,41.840119,-87.618592,Park
2,DOUGLAS,41.843794,-87.617768,CVS pharmacy,41.846859,-87.614555,Pharmacy
3,DOUGLAS,41.843794,-87.617768,Switch Harrisburg,41.843214,-87.618350,Business Service
4,DOUGLAS,41.843794,-87.617768,Metra - 27th Street,41.843938,-87.613268,Train Station


In [21]:
#let's identify the communities with the most venues
District1_venues.groupby('Community Name').count()

,Community Latitude,Community Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Community Name,,,,,,
DOUGLAS,57,57,57,57,57,57
LOOP,100,100,100,100,100,100
NEAR SOUTH SIDE,195,195,195,195,195,195


In [22]:
#let's identify the communities with the most categories of venues. First, let's identify the number of venue categories
print('There are {} uniques venue categories.'.format(len(District1_venues['Venue Category'].unique())))

There are 130 uniques venue categories.


In [ ]:
# let's further analyze each community #

In [23]:
# one hot encoding
District1_onehot = pd.get_dummies(District1_venues[['Venue Category']], prefix="", prefix_sep="")

# add Community Name column back to dataframe
District1_onehot['Community Name'] = District1_venues['Community Name'] 

# move Community Name column to the first column
fixed_columns = [District1_onehot.columns[-1]] + list(District1_onehot.columns[:-1])
District1_onehot = District1_onehot[fixed_columns]

print(District1_onehot.shape)
District1_onehot.head()

(352, 131)


,Community Name,American Restaurant,Art Museum,Arts & Crafts Store,Asian Restaurant,BBQ Joint,Bakery,Bank,Bar,Beer Garden,...,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Train Station,Vegetarian / Vegan Restaurant,Video Store,Wings Joint,Women's Store,Yoga Studio
0,DOUGLAS,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,DOUGLAS,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,DOUGLAS,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,DOUGLAS,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,DOUGLAS,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0


In [24]:
# Next, let's group the communities by the frequency of occurrence of each venue category
District1_grouped = District1_onehot.groupby('Community Name').mean().reset_index()
District1_grouped

,Community Name,American Restaurant,Art Museum,Arts & Crafts Store,Asian Restaurant,BBQ Joint,Bakery,Bank,Bar,Beer Garden,...,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Train Station,Vegetarian / Vegan Restaurant,Video Store,Wings Joint,Women's Store,Yoga Studio
0,DOUGLAS,0.000000,0.00,0.00,0.035088,0.000000,0.035088,0.000000,0.000000,0.000000,...,0.00,0.000000,0.000000,0.000000,0.017544,0.00,0.000000,0.000000,0.000000,0.000000
1,LOOP,0.030000,0.01,0.01,0.010000,0.000000,0.000000,0.010000,0.000000,0.000000,...,0.02,0.000000,0.010000,0.020000,0.000000,0.01,0.000000,0.000000,0.000000,0.010000
2,NEAR SOUTH SIDE,0.010256,0.00,0.00,0.020513,0.010256,0.010256,0.010256,0.010256,0.010256,...,0.00,0.005128,0.005128,0.005128,0.015385,0.00,0.005128,0.010256,0.005128,0.010256


In [35]:
# Let's show the top 5 venue category in each community
num_top_venues = 5

for hood in District1_grouped['Community Name']:
    print("----"+hood+"----")
    temp = District1_grouped[District1_grouped['Community Name'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----DOUGLAS----
                  venue  freq
0                  Park  0.11
1    Chinese Restaurant  0.11
2         Grocery Store  0.07
3           Bus Station  0.07
4  Gym / Fitness Center  0.05


----LOOP----
                  venue  freq
0           Coffee Shop  0.06
1           Pizza Place  0.05
2        Sandwich Place  0.04
3  Gym / Fitness Center  0.04
4   American Restaurant  0.03


----NEAR SOUTH SIDE----
                  venue  freq
0    Chinese Restaurant  0.10
1           Coffee Shop  0.04
2           Pizza Place  0.03
3  Gym / Fitness Center  0.03
4                   Gym  0.03




Result 2: the top 5 venue categories show fewer restaurants (lower frequency) in the LOOP community than the DOUGLAS and NEAR SOUTH SIDE communities, which indicates that the LOOP community may be a good location as the couple's restaurant may face less competion. 

In [ ]:
# let's read the top venues into a dataframe 

In [36]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [37]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Community Name']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
Communities_venues_sorted = pd.DataFrame(columns=columns)
Communities_venues_sorted['Community Name'] =District1_grouped['Community Name']

for ind in np.arange(District1_grouped.shape[0]):
    Communities_venues_sorted.iloc[ind, 1:] = return_most_common_venues(District1_grouped.iloc[ind, :], num_top_venues)

Communities_venues_sorted.head()

,Community Name,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,DOUGLAS,Park,Chinese Restaurant,Grocery Store,Bus Station,Gym / Fitness Center,Food Truck,Deli / Bodega,Pizza Place,New American Restaurant,Dim Sum Restaurant
1,LOOP,Coffee Shop,Pizza Place,Sandwich Place,Gym / Fitness Center,American Restaurant,Boutique,Hotel,Ice Cream Shop,Bookstore,Theater
2,NEAR SOUTH SIDE,Chinese Restaurant,Coffee Shop,Pizza Place,Gym,Sandwich Place,Gym / Fitness Center,Historic Site,Asian Restaurant,Park,Clothing Store


Result 3: the top 10 most common venue analysis show that the NEAR SOUTH SIDE community is an attractive location as it has parks, historic sites and various restaurants, which can appeal to tourists and local residents alike. 

In [ ]:
 # Since there are only 3 unique communities in District 1, there is no need to further segment them. Instead let's group the venue categories into 4 clusters 

In [28]:
# we'll first import the libraries for clustering

from sklearn.cluster import KMeans 
from sklearn.datasets.samples_generator import make_blobs

import matplotlib.pyplot as plt 
%matplotlib inline 
print('Libraries imported.')

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/sklearn/utils/deprecation.py:143: FutureWarning: The sklearn.datasets.samples_generator module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.datasets. Anything that cannot be imported from sklearn.datasets is now part of the private API.
  warnings.warn(message, FutureWarning)


Libraries imported.


In [29]:
# One hot encoding for venue categoies
District1_venues_onehot = pd.get_dummies(District1_venues[['Community Name']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
District1_venues_onehot['Venue Category'] = District1_venues['Venue Category'] 

# move venue category column to the first column
fixed_columns = [District1_venues_onehot.columns[-1]] + list(District1_venues_onehot.columns[:-1])
District1_venues_onehot = District1_venues_onehot[fixed_columns]

print(District1_venues_onehot.shape)
District1_venues_onehot.head()

(352, 4)


,Venue Category,DOUGLAS,LOOP,NEAR SOUTH SIDE
0,Gym / Fitness Center,1,0,0
1,Park,1,0,0
2,Pharmacy,1,0,0
3,Business Service,1,0,0
4,Train Station,1,0,0


In [30]:
District1_venues_grouped = District1_venues_onehot.groupby('Venue Category').mean().reset_index()
print(District1_venues_grouped.shape)
District1_venues_grouped.head() 

(130, 4)


,Venue Category,DOUGLAS,LOOP,NEAR SOUTH SIDE
0,American Restaurant,0.000000,0.600000,0.400000
1,Art Museum,0.000000,1.000000,0.000000
2,Arts & Crafts Store,0.000000,1.000000,0.000000
3,Asian Restaurant,0.285714,0.142857,0.571429
4,BBQ Joint,0.000000,0.000000,1.000000


In [31]:
# set number of clusters to 4
kclusters = 4
District1_venues_clustering = District1_venues_grouped.drop('Venue Category', 1)

# run k-means clustering: 
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(District1_venues_clustering)

# check cluster labels generated for each row in the dataframe: 
kmeans.labels_[0:10]

array([2, 0, 0, 2, 1, 3, 2, 1, 1, 1], dtype=int32)

In [32]:
# Let's add clustering labels to the district 1 venues data
District1_venues_grouped.insert(4, 'Cluster Labels', kmeans.labels_)

In [33]:
# Show the venue categories with the clusters
District1_venues_grouped.head(10)

,Venue Category,DOUGLAS,LOOP,NEAR SOUTH SIDE,Cluster Labels
0,American Restaurant,0.000000,0.600000,0.400000,2
1,Art Museum,0.000000,1.000000,0.000000,0
2,Arts & Crafts Store,0.000000,1.000000,0.000000,0
3,Asian Restaurant,0.285714,0.142857,0.571429,2
4,BBQ Joint,0.000000,0.000000,1.000000,1
5,Bakery,0.500000,0.000000,0.500000,3
6,Bank,0.000000,0.333333,0.666667,2
7,Bar,0.000000,0.000000,1.000000,1
8,Beer Garden,0.000000,0.000000,1.000000,1
9,Big Box Store,0.000000,0.000000,1.000000,1


Result 4: The above clusters show that the LOOP and NEAR SOUTH SIDE Communities stand out as they have art venues such as museum, galleries and craft stores, which can appeal to tourists. In addition, they also have banks indicating potential foot traffic from business customers. 

# Results #

Based on the results 1,2, 3 and 4 John first selected the LOOP and NEAR SOUTH SIDE Communities.

In [ ]:
#For the final selection, John decided to look at the community with the safest elementary school for the couple's child.

In [39]:
selected_schl = schl_features[['DISTRICT','COMMUNITY_AREA_NAME','School_Type', "NAME_OF_SCHOOL", "SAFETY_SCORE", "Longitude", "Latitude"]]
selected_schl = selected_schl[selected_schl.DISTRICT==1 ]
selected_schl = selected_schl[selected_schl.School_Type=='ES']
selected_schl

,DISTRICT,COMMUNITY_AREA_NAME,School_Type,NAME_OF_SCHOOL,SAFETY_SCORE,Longitude,Latitude
267,1,DOUGLAS,ES,John B Drake Elementary School,59.0,-87.617768,41.843794
396,1,NEAR SOUTH SIDE,ES,National Teachers Elementary Academy,67.0,-87.629281,41.852773
483,1,NEAR SOUTH SIDE,ES,South Loop Elementary School,84.0,-87.628651,41.866856
559,1,DOUGLAS,ES,Williams Multiplex Elementary School,NaN,-87.628590,41.844038


Final Result: the table above shows that the "South Loop Elelementary School" is the safest out of the four elementary schools in District 1. Although it is in the NEAR SOUTH SIDE community, its name indicates it is very close to the LOOP community. Hence John recommended a location close to the shouth loop elementary school as the first choice, then the National Teachers Elementary Academy and the John B Drake Elementary Schools as the second and third choice, respectively.

## Discussion ##

This analysis did not analyze the culinary taste and preferences of the tourists and business employees in these three communities considered. In addition, this analysis did not consider commercial space availability suitable for restaurant. These are some of the information that can improve this analysis.

## Conclusion ##

In this analysis, John started with the requirements listed by his client, then he collected the relevant data, designed and implemented a mehtodology that analyzed several aspects of Chicago communities accorddingly.While the analytical methodology can be further improved with target population preferences and availability of commercial space for lease, John's final recommendation meets his client requirements in terms of neighborhood safety, proximity to the couple residence, convenient commute and their child's school needs. 